<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<font size='1'>&copy; Modificado en 2017 por el cuerpo Docente IIC2233015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>

</p>

En Python, hemos visto que podemos guardar datos de distinto tipo dentro de variables.

Por ejemplo, enteros, *strings*, listas y *floats*

In [1]:
a = 13
b = "hola"
c = [None, 2, "chao"]
d = 1.0

Como vimos la materia de decoradores, también podemos guardar funciones en variables.

In [2]:
def suma(a, b):
    return a + b

variable = suma
variable(1, 2)

3

Un aspecto que hasta ahora no hemos visto, es que también podemos guardar clases en variables.

In [3]:
class MiClaseNormal: 
    
    def __init__(self, atributo):
        self.atributo = atributo

variable2 = MiClaseNormal

In [4]:
instancia_de_mi_clase_normal = variable2(2)
print(instancia_de_mi_clase_normal.atributo)

2


Todos las cosas que se pueden guardar en variables, también se pueden pasar como argumentos a una función. Ya vimos que podíamos pasar funciones como argumentos. Veamos cómo hacer lo mismo con una clase.

In [5]:
def imprimir_tipo_de_dato(arg):
    print(type(arg))

In [6]:
imprimir_tipo_de_dato(1)
imprimir_tipo_de_dato(suma)
imprimir_tipo_de_dato(instancia_de_mi_clase_normal)
imprimir_tipo_de_dato(MiClaseNormal)

<class 'int'>
<class 'function'>
<class '__main__.MiClaseNormal'>
<class 'type'>


Todos estos elementos que se pueden guardar en variables y pasar como argumentos tienen un tipo de dato, es decir, son instancias de una clase.

Por ejemplo, los enteros son instancias de la clase `int`, las funciones son instancias de la clase `function` y los objetos que creamos usando MiClaseNormal con instancias de `MiClaseNormal`.

Ahora, como vimos que las clases también se pueden guardar en variables y ser pasadas como argumento, también deberían ser instancias de una clase. Las clases que definen el tipo de dato de otras clases, se llaman en Python **Metaclases**.

El comando `type()` retorna el tipo de dato, es decir la clase, de una instancia. Si esa instancia es una clase —es decir un tipo de datos—, entonces su `type()` retornará cuál es la metaclase de esa clase.

En este ejemplo, podemos ver que la clase MiClase es de tipo `type`. En otras palabras `type` es la Metaclase de MiClase.

In [7]:
type(MiClaseNormal)

type

La metaclase de la mayoría de clases es `type`, pero también podemos crear nuestras propias metaclases.

In [8]:
a =1
a.__class__

int

In [9]:
type(a)

int

# Metaclases

Una metaclase es una *clase* cuyas instancias son *clases* en vez de *objetos*. Básicamente las metaclases son tipos de clases, así como las clases son tipos de objetos. Los objetos que creamos a través de una metaclase tienen la particularidad de ser capaces de crear otros objetos o instancias.

![](img/metaclass.png)

## Cómo asignar un tipo a una clase

Para declarar que una clase tiene un cierto tipo, debemos agregar el keyworded argument `metaclass={tipo_de_clases}`. 

In [10]:
class ClaseEjemplo(metaclass=type):
    pass

Durante la creación de la clase, Python pregunta: ¿está el keyword `metaclass` dentro de los argumentos en la clase `ClaseEjemplo`? Si el argumento existe, entonces creará en memoria una clase con el nombre `ClaseEjemplo` usando lo que hay en `metaclass`. En caso contrario, Python usará la misma metaclase de la clase padre, para crear la nueva clase.

La variable `metaclass` debe tener *algo* que pueda crear un clase, es decir, debe tener una *metaclase*. En Python, las metaclases de declaran como clases que heredan de `type`. 

## Metaclases personalizadas

El principal propósito de las metaclases es modificar una clase cuando es creada. Para controlar la creación
e inicialización de una clase en la metaclase podemos implementar los métodos `__new__` e `___init__`, respectivamente, en la metaclase. 

`__new__` debe ser implementado cuando queremos controlar la creación de una nueva clase. Utilizando este método podemos cambiar el nombre de la clase y las clases de las cuáles hereda. También podemos modificar sus atributos y métodos. En el siguiente ejemplo pueden ver que podeos cambiar básicamente cualquier cosa utilizando el `__new__`.


In [28]:
class MiMetaClase(type):
    def __new__(meta, nombre, base_clases, diccionario):
        print ('-----------------------------------')
        print("Creando la clase.. {} ".format(nombre))
        print(meta)
        print(base_clases)
        # supongamos que siempre obligamos a la clase a tener este atributo
        diccionario.update(dict({'atributo_obligatorio': 10}))
        nombre = "Meta_" + nombre # Además le cambiamos el nombre
        print(diccionario)
        # Aquí llamaremos al método __new__ de "type", después de haber hecho las modificaciones que queríamos
        # Este método __new__ de type es lo que se habría llamado si no hubiésemos usado la metaclase personalizada
        return super().__new__(meta, nombre, base_clases, diccionario)

In [29]:
class MiClase(metaclass = MiMetaClase): # Declaramos que la clase MiClase es del tipo MiMetaclase
    mi_parametro = 4
    hola = "chupa"
    
    def func(self, params):
        pass

-----------------------------------
Creando la clase.. MiClase 
<class '__main__.MiMetaClase'>
()
{'__module__': '__main__', '__qualname__': 'MiClase', 'mi_parametro': 4, 'hola': 'chupa', 'func': <function MiClase.func at 0x000002606AD981E0>, 'atributo_obligatorio': 10}


In [30]:
m1 = MiClase()
print(m1.atributo_obligatorio) # Recuerden que este atributo es de clase
print(m1.__class__.__name__) # Recuerden que le cambiamos el nombre arriba
print(isinstance(MiClase, MiMetaClase)) # MiClase es una instancia de MiMetaClase
print(isinstance(m1, MiMetaClase)) # Esto es falso porque m1 es una instancia de MiClase, no de MiMetaClase

10
Meta_MiClase
True
False


`__init__` debería ser implementado cuando queremos controlar la inicialización de la clase una vez que ya está creado. En este método no se pueden modificar ni el nombre ni las clases bases. Sí se puede agregar funciones y atributos a la clase.

In [31]:
class MiMetaClase(type):
    def __new__(meta, nombre, base_clases, diccionario):
        print ('-----------------------------------')
        print("Creando la clase: {} ".format(nombre))
        print(meta)
        print(base_clases)
        diccionario.update({'atributo_obligatorio': 10})
        nombre = "Meta_" + nombre
        print(diccionario)
        base_clases = (int,)
        print(base_clases)
        return super().__new__(meta, nombre, base_clases, diccionario)
    
    def __init__(cls, nombre, base_clases, diccionario):
        # diccionario mantiene los cambios realizados en el new, porque mantuvimos la referencia. 
        # Nombre y base_clases, son de tipo str y tuple que no son mutables, por lo que cuando los
        # cambiamos la referencia cambió.
        print ('-----------------------------------')
        print("Inicializando la clase: {} ".format(nombre))
        print(cls)
        print(cls.__mro__)
        print(base_clases) # La misma que 
        print(diccionario)
        # La clase ya fue creada así que cualquier cambio a los parámetros nombre, bases_clases y diccionario
        # no generará ningún cambio. Para realizar cambios debemos hacerlos directamente en la clase
        diccionario.update({'atributo_obligatorio': 15})
        nombre = "Meta_" + nombre 
        
        # Esto sí se verá reflejado en la clase
        cls.atributo_init = 20
        setattr(cls, 'atributo_init_2', '35')
        
        return super().__init__(nombre, base_clases, diccionario) # En estricto rigor no es necesario retornar

In [32]:
class MiClase(metaclass = MiMetaClase): # Declaramos que la clase MiClasse es del tipo MiMetaclase
    mi_parametro = 4
    def func(self, params):
        pass

-----------------------------------
Creando la clase: MiClase 
<class '__main__.MiMetaClase'>
()
{'__module__': '__main__', '__qualname__': 'MiClase', 'mi_parametro': 4, 'func': <function MiClase.func at 0x000002606AD8E400>, 'atributo_obligatorio': 10}
(<class 'int'>,)
-----------------------------------
Inicializando la clase: MiClase 
<class '__main__.MiClase'>
(<class '__main__.MiClase'>, <class 'int'>, <class 'object'>)
()
{'__module__': '__main__', '__qualname__': 'MiClase', 'mi_parametro': 4, 'func': <function MiClase.func at 0x000002606AD8E400>, 'atributo_obligatorio': 10}


In [16]:
m1 = MiClase()
print(m1.atributo_obligatorio) # Recuerden que este atributo es de clase. Tiene el mismo valor que en el new
print(m1.atributo_init) # Este atributo es de clase y fue creado en el init
print(m1.atributo_init_2) # Este atributo es de clase y fue creado en el init
print(m1.__class__.__name__) # Le cambiamos el nombre arriba
print(isinstance(MiClase, MiMetaClase)) # MiClase es una instancia de MiMetaClase
print(isinstance(m1, MiMetaClase)) # Esto es falso porque m1 es una instancia de MiClase, no de MiMetaClase

10
20
35
Meta_MiClase
True
False


### Creación de instancias de una clase

Cada vez que hacemos `variable()` estamos llamando al método `__call__`. Se dice que un objeto es *callable* cuando tiene el método `__call__` implementando. En el caso de metaclases, el método `__call__` es especial, ya que es ejecutado cuando una *clase* es llamada para crear una instancia de un nuevo objeto. Cada vez que han instanciado objetos de una clase usando los paréntesis, como `list()`, lo que en verdad han hecho es llamar al método `__call__`. 

En una metaclase podemos modificar el método `__call__` para que sea posible intersectar la creación de instancias. 

In [17]:
class MiMetaClase(type):
    def __call__(cls, *args, **kwargs):
        """
        __call__ recibe cls que es el objeto que corresponde a la clase
        que se está llamando, y *args/**args los argumentos que esta recibe
        durante su instanciacion. 
        """
        print("__call__ of  {}".format(str(cls)))
        print("__call__ *args= {}".format(str(args)))
        return super().__call__(*args, **kwargs)

class MiClase(metaclass = MiMetaClase):
    def __init__(self, a, b):
        print('Objeto de MiClase con a={}, b={}'.format(a, b))


ob1 = MiClase(1, 2)

__call__ of  <class '__main__.MiClase'>
__call__ *args= (1, 2)
Objeto de MiClase con a=1, b=2


## Ejemplos de metaclases
### Clase que no puede ser subclaseada

Definamos una metaclase que asegure que ninguna clase va a poder heredar de la metaclase. Se podrán crear clases que tengan la metaclase `MetaFinal` pero no será posible que estas clases sean la clase base de otras.

In [34]:
class MetaFinal(type):
    def __new__(mcl, name, bases, classdict):
        """
        Este paso chequea que la clase b, de la que se quiere heredar,
        no sea una instancia de MetaFinal.
        """
        for b in bases:
            if isinstance(b, mcl): # Si b fue creada con mcl, entonces la clase {name} no puede ser creada
                raise TypeError("type '{0}' is not an acceptable base type".format(b.__name__))
        return super().__new__(mcl, name, bases, dict(classdict))

    
class C(metaclass=MetaFinal): 
    pass


print(isinstance(C, MetaFinal)) # No tenemos problemas porque C no tiene clases bases que hereden de MetaFinal


class D(C): # Aquí tendremos un error
    pass

<class '__main__.MetaFinal'>
True
<class '__main__.MetaFinal'>


TypeError: type 'C' is not an acceptable base type

### Singletón

En este ejemplo veremos como definir una metaclase que asegure que cada vez que se llama a la clase para crear una nueva instancia, ésta retorne la nueva instancia, si y sólo si, ninguna instancia de la clase se ha creado antes. En otro caso, se debería retornar la misma instancia creada la primera vez:

In [19]:
class MetaSoloUna(type):
    def __call__(cls, *args, **kw):
        if not hasattr(cls, "instance"): # Recuerda que siempre podemos agregar variables en tiempo de ejecución
             cls.instance = super().__call__(*args, **kw)
        return cls.instance
    
class SoloUna(metaclass=MetaSoloUna):
    pass


a = SoloUna()
print(a.instance == None)
b = SoloUna()
print(a is b)
print(id(a))
print(id(b))


False
True
139842297388672
139842297388672


## ¿Cuál es la diferencia entre los métodos de la metaclase y la clase?

In [35]:
class Meta(type):
    def __new__(meta, nombre, base_clases, diccionario):
        print('\nMeta - __new__')
        print(meta)
        print(nombre)
        print(base_clases)
        print(diccionario)
        return super().__new__(meta, nombre, base_clases, diccionario)
    
    def __init__(cls, nombre, base_clases, diccionario):
        print('\nMeta - __init__')
        print(cls)
        print(nombre)
        print(base_clases)
        print(diccionario)
        return super().__init__(nombre, base_clases, diccionario) # En estricto rigor no es necesario retornar
    
    def __call__(cls, *args, **kwargs):
        print('\nMeta - __call__')
        print(cls)
        print(args)
        print(kwargs)
        return super().__call__(*args, **kwargs)

class A(metaclass=Meta):

    def __new__(cls, *args, **kwargs):
        print('\nClass - __new__')
        print(cls)
        print(args)
        print(kwargs)
        return super().__new__(cls)

    def __init__(self, *args, **kwargs):
        print("\nClass - __init__")
        print(args)
        print(kwargs)
    
    def __call__(self, *args, **kwargs):
        print('\nClass - __call__')
        print(args)
        print(kwargs)



Meta - __new__
<class '__main__.Meta'>
A
()
{'__module__': '__main__', '__qualname__': 'A', '__new__': <function A.__new__ at 0x000002606AD988C8>, '__init__': <function A.__init__ at 0x000002606AD98840>, '__call__': <function A.__call__ at 0x000002606AD98158>, '__classcell__': <cell at 0x000002606AD00108: empty>}

Meta - __init__
<class '__main__.A'>
A
()
{'__module__': '__main__', '__qualname__': 'A', '__new__': <function A.__new__ at 0x000002606AD988C8>, '__init__': <function A.__init__ at 0x000002606AD98840>, '__call__': <function A.__call__ at 0x000002606AD98158>, '__classcell__': <cell at 0x000002606AD00108: Meta object at 0x00000260691BFB78>}


In [21]:
a = A(1, 2, 'varstr', kworded_var='var')
a(9, 4, kworded_var='var')


Meta - __call__
<class '__main__.A'>
(1, 2, 'varstr')
{'kworded_var': 'var'}

Class - __new__
<class '__main__.A'>
(1, 2, 'varstr')
{'kworded_var': 'var'}

Class - __init__
(1, 2, 'varstr')
{'kworded_var': 'var'}

Class - __call__
(9, 4)
{'kworded_var': 'var'}


Esta tabla resume las diferencias entre lo que hacen los métodos de la clase y de la metaclase.


Métodos     | Clase                       | MetaClase
---------- | --------------------------- | -------------------------------------
**`__new__`**  | `cls`, `args`, `kwargs`     | `meta`, `name`, `bases`, `attributes`
Retorna    | objeto de la clase `cls`    | clase del tipo `meta`, nombre `name` que hereda de `bases`, y que tiene `attributes` atributos
 | | 
**`__init__` **| `self` (creado por `new`), `args`, `kwargs`  | `cls` (creado por el `new`), `name`, `bases`, `attributes`
Retorna    | nada                      | nada, o clase `cls` inicializada
 | | 
**`__call__` **| `self`, `args`, `kwargs`    | `cls`, `args`, `kwargs`
Retorna    | no está obligado a retornar | un objeto de la clase `cls` creado e inicializado con `args` y `kwargs`
